In [ ]:
import requests, pandas as pd

base = "https://archive-api.open-meteo.com/v1/archive"  # 同样用 archive-api
params = {
    "latitude":  30.869,  #湖州纬度
    "longitude": 120.093,  #湖州经度
    "start_date":"2025-05-01",
    "end_date":  "2025-06-16",
    "hourly": ",".join([
        "temperature_2m",
        "relativehumidity_2m",
        "shortwave_radiation",
        "direct_radiation",
        "diffuse_radiation",
        "precipitation",
        "windspeed_10m"
    ]),
    "timezone":"Asia/Shanghai",
    "models":"best_match"  
}

r = requests.get(base, params=params)
r.raise_for_status()
js = r.json()

df = pd.DataFrame(js["hourly"])
df["ds"] = pd.to_datetime(df.pop("time"))
print(df.head())

In [ ]:

import matplotlib.pyplot as plt
from nixtlats import NixtlaClient
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
client = NixtlaClient(api_key="nixak-JIoA4Vfr07t9QKnD833qhMqnI33AtmdC7Y4T8OSiHIbAm5CCoVpDnu2X9il3b45SkPdxoRsBeuXrhmjB")
import pandas as pd, numpy as np

In [ ]:
# 5 月
may = pd.read_csv(
    "1小时湖州光伏发电功率_2025_05.csv",
    usecols=["时间", "Grid_YC16512(逆变器组发电功率)"]
)
may = may.rename(columns={
    "时间": "ds",
    "Grid_YC16512(逆变器组发电功率)": "y"
})
may["ds"] = pd.to_datetime(may["ds"])

# 6 月
june = pd.read_csv(
    "1小时湖州光伏发电功率_2025_06.csv",
    usecols=["时间", "Grid_YC16512(逆变器组发电功率)"]
)
june = june.rename(columns={
    "时间": "ds",
    "Grid_YC16512(逆变器组发电功率)": "y"
})
june["ds"] = pd.to_datetime(june["ds"])

In [ ]:
weather_df = df.copy()

weather_cols = [
    "temperature_2m",
    "relativehumidity_2m",
    "shortwave_radiation",
    "direct_radiation",
    "diffuse_radiation",
    "precipitation",
    "windspeed_10m"
]

# —— 3️⃣ 合并发电与天气 ——  
may_w  = pd.merge(may,  weather_df, on="ds", how="left")
june_w = pd.merge(june, weather_df, on="ds", how="left")

# 对极少 NaN 做插值或前向填充
may_w[weather_cols]  = may_w[weather_cols].interpolate().bfill()
june_w[weather_cols] = june_w[weather_cols].interpolate().bfill()


In [ ]:
# 预测步长 = 6 月数据点数
h = len(june)
# 测试期天气外生特征
X_test = june_w[["ds"] + weather_cols].copy()


# single-series 接口需要 unique_id
for df_ in (may_w, june_w, X_test):
    df_["unique_id"] = 1

forecast = client.forecast(
    df=may_w,            # 历史 ds, y, weather_cols, unique_id
    X_df=X_test,         # 未来期 ds, weather_cols, unique_id
    id_col="unique_id",
    h=h,
    freq="h",            # 小写 'h'
    time_col="ds",
    target_col="y",
    finetune_steps=50
)

In [ ]:
result = june.rename(columns={"y": "Actual"})
result["Forecast"] = forecast["TimeGPT"].values

# 误差

#mae  = mean_absolute_error(result["Actual"], result["Forecast"])
#rmse = np.sqrt(mean_squared_error(result["Actual"], result["Forecast"]))
#print(f"MAE:  {mae:.3f}")
#print(f"RMSE: {rmse:.3f}")

# 画图
plt.figure(figsize=(12,4))
plt.plot(result.ds, result.Actual,  label="Actual (June)",  alpha=0.7)
plt.plot(result.ds, result.Forecast, label="Forecast (June)", alpha=0.7)
plt.title("Huzhou, Train May → Forecast June (with finetune = 50)")
plt.xlabel("Time"); plt.ylabel("Generated Power /kW")
plt.legend(); plt.grid(True); plt.tight_layout()
plt.show()